In [3]:
# Libraries
import numpy as np
import pandas as pd
import math

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [4]:
# Running data - current day and next day
dat = pd.read_excel('running_data.xlsx')

In [5]:
dat.shape

(1972, 5)

In [6]:
dat.head()

,date,run_dist,notes,run_dist_1,notes_1
171,2016-10-10,10.0,"AM: Warmup (2 miles), 2 sets of (80, 50, 50), ...",10.0,"AM: Warmup (1.5 miles), Lift, Cooldown (2 mile..."
225,2016-11-21,10.0,"AM: Warmup (2 miles), 8x150 (Extensive Tempo),...",8.0,"AM: Warmup (1.5 miles), Lift, Cooldown (1 mile..."
226,2016-11-22,8.0,"AM: Warmup (1.5 miles), Lift, Cooldown (1 mile...",11.0,"AM: Warmup (2.5 miles), 8x300 with 2:30 rest (..."
227,2016-11-23,11.0,"AM: Warmup (2.5 miles), 8x300 with 2:30 rest (...",8.0,8 mile run. Not able to do squats or RDLs.
228,2016-11-24,8.0,8 mile run. Not able to do squats or RDLs.,10.5,"AM: Warmup (2 miles), 5x800 with 1:45 rest (Go..."


### Test on small dataset

In [7]:
dat_small = pd.read_excel('rd_small_full.xlsx')

In [8]:
print dat.shape
dat_small.head()

(1972, 5)


,date,run_dist,notes,is_workout,difficulty,run_dist_1,notes_1
0,2016-12-29,8.00,"AM: 5 miles; PM: Warmup, lift, cooldown",0,0,9.0,"AM: Warmup (2 miles), 3x1000 with 3:30 recover..."
1,2015-07-26,10.00,Felt a little sore at the beginning of the run...,0,3,6.5,"6 mile run at around 6:40 pace, 6x100 strides."
2,2015-07-28,7.50,AM: 3 mile run at around 6:40 pace; PM: Warm-u...,1,2,5.0,5 mile run at around 6:30 pace.
3,2015-08-19,6.00,"6 mile run at 6:40 pace, 4x100 strides",0,3,7.0,"7 mile run at 6:45 pace, 4x100 strides"
4,2015-10-18,10.25,"10 mile run in 61:00 at Minuteman, 4x100 strid...",0,1,8.0,"AM: 3 miles; PM: Warmup (1.5 miles), 7x120 at ..."


In [9]:
dat_small.iloc[0]

date                                        2016-12-29 00:00:00
run_dist                                                      8
notes                   AM: 5 miles; PM: Warmup, lift, cooldown
is_workout                                                    0
difficulty                                                    0
run_dist_1                                                    9
notes_1       AM: Warmup (2 miles), 3x1000 with 3:30 recover...
Name: 0, dtype: object

In [10]:
vectorizer = CountVectorizer(ngram_range=(1,2))
X_feats = vectorizer.fit_transform(dat_small['notes'])

In [11]:
X_feats.shape

(407, 9575)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X_feats, dat_small['difficulty'], test_size=0.25)

In [13]:
mnb = MultinomialNB()
nb_model = mnb.fit(X_train, y_train)
y_pred = nb_model.predict(X_test)
#y_pred = mnb.fit(X_train, train['label']).predict(X_test)
print("Correct labels out of %d points : %d"
      % (X_test.shape[0],(y_test == y_pred).sum()))
print 'Accuracy:', float(sum(y_test == y_pred))/X_test.shape[0]
print 'Within 1:', float(sum((abs(y_test - y_pred) <= 1)))/X_test.shape[0]

Correct labels out of 102 points : 44
Accuracy: 0.43137254902
Within 1: 0.754901960784


In [14]:
results = pd.DataFrame(data = {'pred':y_pred, 'true':y_test}) #'diff':[abs(pred-true) for pred, true in zip(y_pred,y_test)]})

In [15]:
results.shape

(102, 2)

In [16]:
results[results['pred'] == 0]

,pred,true


In [17]:
res = results[(results['true'] != 0) & (results['pred'] != 0)]

In [18]:
print 'Accuracy:', float(sum(res['pred'] == res['true']))/res.shape[0]
print 'Within 1:', float(sum((abs(res['pred'] - res['true']) <= 1)))/res.shape[0]

Accuracy: 0.458333333333
Within 1: 0.78125


In [19]:
y_probs = nb_model.predict_proba(X_test)

In [20]:
y_probs[0]

array([  1.37402999e-05,   1.84002363e-01,   1.26602854e-04,
         3.32244460e-05,   8.13721109e-01,   2.10296046e-03])

In [21]:
y_test.iloc[0]

0

In [22]:
i = 164
print dat_small['difficulty'].iloc[i]
dat_small['notes'].iloc[i]

5


u"Day off, feeling a bit sick (don't do too well with travel), so day off was good."

In [23]:
X_test

<102x9575 sparse matrix of type '<type 'numpy.int64'>'
	with 6236 stored elements in Compressed Sparse Row format>

In [35]:
accuracy = [None]*20
accuracy_within_1 = [None]*20
for j in range(20):  
    X_train, X_test, y_train, y_test = train_test_split(X_feats, dat_small['difficulty'], test_size=0.25, random_state=j)
    mnb = MultinomialNB()
    nb_model = mnb.fit(X_train, y_train)
    y_pred = nb_model.predict(X_test)
    #y_pred = mnb.fit(X_train, train['label']).predict(X_test)
    print 'Accuracy:', float(sum((y_test == y_pred)))/X_test.shape[0]
    accuracy[j] = float(sum((y_test == y_pred)))/X_test.shape[0]
    accuracy_within_1[j] = float(sum((abs(y_test - y_pred) <= 1)))/X_test.shape[0]

Accuracy: 0.323529411765
Accuracy: 0.303921568627
Accuracy: 0.392156862745
Accuracy: 0.401960784314
Accuracy: 0.254901960784
Accuracy: 0.382352941176
Accuracy: 0.352941176471
Accuracy: 0.441176470588
Accuracy: 0.362745098039
Accuracy: 0.372549019608
Accuracy: 0.450980392157
Accuracy: 0.323529411765
Accuracy: 0.372549019608
Accuracy: 0.372549019608
Accuracy: 0.343137254902
Accuracy: 0.352941176471
Accuracy: 0.372549019608
Accuracy: 0.421568627451
Accuracy: 0.43137254902
Accuracy: 0.441176470588


In [36]:
print float(sum(accuracy))/len(accuracy)
print float(sum(accuracy_within_1))/len(accuracy_within_1)

0.373529411765
0.749019607843


In [37]:
dat.head()

,date,run_dist,notes,run_dist_1,notes_1
171,2016-10-10,10.0,"AM: Warmup (2 miles), 2 sets of (80, 50, 50), ...",10.0,"AM: Warmup (1.5 miles), Lift, Cooldown (2 mile..."
225,2016-11-21,10.0,"AM: Warmup (2 miles), 8x150 (Extensive Tempo),...",8.0,"AM: Warmup (1.5 miles), Lift, Cooldown (1 mile..."
226,2016-11-22,8.0,"AM: Warmup (1.5 miles), Lift, Cooldown (1 mile...",11.0,"AM: Warmup (2.5 miles), 8x300 with 2:30 rest (..."
227,2016-11-23,11.0,"AM: Warmup (2.5 miles), 8x300 with 2:30 rest (...",8.0,8 mile run. Not able to do squats or RDLs.
228,2016-11-24,8.0,8 mile run. Not able to do squats or RDLs.,10.5,"AM: Warmup (2 miles), 5x800 with 1:45 rest (Go..."


In [49]:
X_train_diff, X_test_diff, y_train_diff, y_test_diff = train_test_split(dat_small['notes'], dat_small['difficulty'], test_size=0.25)
X_train_work, X_test_work, y_train_work, y_test_work = train_test_split(dat_small['notes'], dat_small['is_workout'], test_size=0.25)

nb_model_diff = Pipeline([('vect', CountVectorizer(ngram_range=(1,2))), 
                     ('tfidf', TfidfTransformer()), ('clf', MultinomialNB()),])
nb_model_work = Pipeline([('vect', CountVectorizer(ngram_range=(1,2))), ('clf', MultinomialNB()),])
nb_model_diff.fit(X_train_diff, y_train_diff)
nb_model_work.fit(X_train_work, y_train_work)
#print(nb_model_diff.predict_proba(X_test_diff))
pred_diff = nb_model_diff.predict(X_test_diff)
pred_work = nb_model_work.predict(X_test_work)
print float(sum(y_test_diff == pred_diff))/X_test_diff.shape[0]
print float(sum((abs(y_test_diff - pred_diff) <= 1)))/X_test_diff.shape[0]
print float(sum(y_test_work == pred_work))/X_test_work.shape[0]

0.225490196078
0.509803921569
0.862745098039


In [50]:
dat.head()

,date,run_dist,notes,run_dist_1,notes_1
171,2016-10-10,10.0,"AM: Warmup (2 miles), 2 sets of (80, 50, 50), ...",10.0,"AM: Warmup (1.5 miles), Lift, Cooldown (2 mile..."
225,2016-11-21,10.0,"AM: Warmup (2 miles), 8x150 (Extensive Tempo),...",8.0,"AM: Warmup (1.5 miles), Lift, Cooldown (1 mile..."
226,2016-11-22,8.0,"AM: Warmup (1.5 miles), Lift, Cooldown (1 mile...",11.0,"AM: Warmup (2.5 miles), 8x300 with 2:30 rest (..."
227,2016-11-23,11.0,"AM: Warmup (2.5 miles), 8x300 with 2:30 rest (...",8.0,8 mile run. Not able to do squats or RDLs.
228,2016-11-24,8.0,8 mile run. Not able to do squats or RDLs.,10.5,"AM: Warmup (2 miles), 5x800 with 1:45 rest (Go..."


In [51]:
is_workout = nb_model_work.predict(dat['notes'])
is_workout_1 = nb_model_work.predict(dat['notes_1'])
difficulty = nb_model_diff.predict(dat['notes'])
difficulty_1 = nb_model_diff.predict(dat['notes_1'])

In [52]:
dat_knn = pd.DataFrame({'run_dist':dat['run_dist'], 'difficulty':difficulty, 'is_workout':is_workout,
                        'run_dist_1':dat['run_dist_1'], 'difficulty_1':difficulty_1, 'is_workout_1':is_workout_1})

In [54]:
dat_knn.iloc[120:130]

,difficulty,difficulty_1,is_workout,is_workout_1,run_dist,run_dist_1
257,1,3,1,0,5.5,5.0
258,3,1,0,0,5.0,7.0
259,1,1,0,1,7.0,7.5
260,1,3,1,0,7.5,10.0
263,3,1,0,1,10.0,7.5
264,1,1,1,1,7.5,7.0
265,1,1,1,1,7.0,9.0
266,1,0,1,0,9.0,0.0
267,0,1,0,1,0.0,5.5
268,1,1,1,0,5.5,8.5


In [55]:
dat_knn.shape

(1972, 6)

In [307]:
# Write file to xlsx -- DO NOT RUN THIS AGAIN--IT WILL OVERWRITE FILE
#dat_knn.to_excel('dat_knn_2.xlsx')